# Frequency

In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd

feature_pos = [[0,8],[8,12],[12,28],[28,44],[44,52],[52,64],[64,80],[80,92],[92,102]] 
df_train = pd.read_csv('train_kaggle.csv')
df_test = pd.read_csv('sample_solution.csv')
Y = df_train['Label'].values

## Save Frequency Features
Skip this if you have saved the features

In [3]:
import numpy as np

dataframes = np.load('allData.npy', allow_pickle = True)
dataframes.shape

def load_test_dataframe(id):
    test_data = np.load("test/test/{}.npy".format(id))
    return pd.DataFrame(data=test_data)

testdatas = []
for id in df_test['Id']:
    dfi = load_test_dataframe(id)
    testdatas.append(dfi.values)

In [4]:
feat = dataframes[0].shape[1]
# feat = 12

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

def one_hot_encoding(encoder, dfs, col):
    if encoder == None:
        data = set(dfs[0][:,col])
        for i in range(1,len(dfs)):
            df = dfs[i]
            data |= set(df[:,col])
        encoder = OneHotEncoder(handle_unknown='ignore')
        encoder = encoder.fit([[str(e)] for e in data])
    encodedData = []
    for df in dfs:
        mat = np.sum(encoder.transform([[str(e)] for e in df[:,col]]), axis=0)
        mat = np.squeeze(np.asarray(mat))
        encodedData.append(mat)
    encodedData = np.stack(encodedData)    
    return encoder, encodedData

def one_hot_encoding_feature(encoders, dfs, feature):
    if len(encoders) == 0:
        encoder, data = one_hot_encoding(None, dfs, feature_pos[feature][0])
        encoders.append(encoder)    
        for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
            encoder, dataCol = one_hot_encoding(None, dfs, col)
            encoders.append(encoder)
            data = np.concatenate((data, dataCol), axis=1)
    else:
        encoder, data = one_hot_encoding(encoders[0], testdatas, feature_pos[feature][0])
        for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
            i = col - feature_pos[feature][0]
            encoder, dataCol = one_hot_encoding(encoders[i], testdatas, col)
            data = np.concatenate((data, dataCol), axis=1)
    return encoders, data

In [ ]:
import gc

# Give up last feature
for feat in range(len(feature_pos) - 1):
    print(feat)
    encoders, train = one_hot_encoding_feature([], dataframes, feat)
    encoders, test = one_hot_encoding_feature(encoders, testdatas, feat)
    np.save('freq/' + str(feat) + '_train.npy', train)
    np.save('freq/' + str(feat) + '_test.npy', test)
    gc.collect()

In [5]:
from tqdm import tqdm

def handle_integers(dfs, col):
    data = []
    for df in dfs:
        sumCol = np.sum(df[:,col], axis=0).reshape(1, -1)
        meanCol = np.mean(df[:, col], axis=0).reshape(1, -1)
        # stdCol = np.mean(df[:, col]).reshape(1, -1)
        dataCol = np.concatenate([sumCol, meanCol], axis=1)
        # dataCol = sumCol
        # print(dataCol.shape)
        data.append(dataCol)
    data = np.concatenate(data, axis=0)    
    # print(data.shape)
    return data

# Extract based on correlations
select_int_col = [True, True, True, True, True, True, False, True, False, False]
def handle_integers_feature(dfs, feature):
    if feature != 8:
        return None
    data = handle_integers(dfs, feature_pos[feature][0])
    for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
        i = col - feature_pos[feature][0]
        if select_int_col[i] == False:
            continue
        dataCol = handle_integers(dfs, col)
        data = np.concatenate((data, dataCol), axis=1)
    return data

train = handle_integers_feature(dataframes, 8)
test = handle_integers_feature(testdatas, 8)
np.save('freq/8_train.npy', train)
np.save('freq/8_test.npy', test)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00,  9.42it/s]


## Load Frequency Features

In [3]:
import numpy as np
feature_selected = [True,True,True,True,True,True,True,True,True] 

def load_freq(feat, postfix):
    data = np.load("freq/{}_{}.npy".format(feat, postfix))
    return data

def load_test_freq(feat):
    return load_freq(feat, 'test')

def load_train_freq(feat):
    return load_freq(feat, 'train')

In [4]:
def select_features(feature_selected):
    XTrain = []
    XTest = []
    for feat in range(len(feature_pos)):
        if feature_selected[feat] == False:
            continue
        train = load_train_freq(feat)
        test = load_test_freq(feat)
        XTrain.append(train)
        XTest.append(test)
    XTrain = np.concatenate(XTrain, axis=1)
    XTest = np.concatenate(XTest, axis=1)
    return XTrain, XTest

def all_not_selected(feature_selected):
    for i in range(len(feature_pos)):
        if feature_selected[i] == True:
            return False
    return True

In [11]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn import metrics
import gc

scores = [0]
f0s = [0]
f1s = [0]
f2s = [0]
f3s = [0]
f4s = [0]
f5s = [0]
f6s = [0]
f7s = [0]
score = (0, 0, 0, 0, 0, 0, 0, 0, 0)
cnt = 0
for f0 in range(2):
    feature_selected[0] = bool(f0)
    for f1 in range(2):
        feature_selected[1] = bool(f1)
        for f2 in range(2):
            feature_selected[2] = bool(f2)
            for f3 in range(2):
                feature_selected[3] = bool(f3)
                for f4 in range(2):
                    feature_selected[4] = bool(f4)
                    for f5 in range(2):
                        feature_selected[5] = bool(f5)
                        for f6 in range(2):
                            feature_selected[6] = bool(f6)
                            for f7 in range(2):
                                cnt += 1
                                feature_selected[7] = bool(f7)
                                if all_not_selected(feature_selected) == True:
                                    continue
                                if cnt % 10 == 0:
                                    print(cnt)
                                XTrain, XTest = select_features(feature_selected)
                                del XTest
                                XTrain, X_valid, y_train, y_valid = train_test_split(XTrain, Y, test_size=0.2, random_state=42)
                                gc.collect()
                                model = LGBMClassifier()
                                model.fit(XTrain, y_train)
                                pred = model.predict_proba(X_valid)[:,1]
                                fpr, tpr, thresholds = metrics.roc_curve(y_valid, pred)
                                auc_score = metrics.auc(fpr, tpr)
                                scores.append(auc_score)
                                f0s.append(f0)
                                f1s.append(f1)
                                f2s.append(f2)
                                f3s.append(f3)
                                f4s.append(f4)
                                f5s.append(f5)
                                f6s.append(f6)
                                f7s.append(f7)
                                gc.collect()
                                if score[0] < auc_score:
                                    score = (auc_score, f0, f1, f2, f3, f4, f5, f6, f7)

KeyboardInterrupt: 

In [17]:
auc_csv = pd.DataFrame({'f0': f0s, 'f1': f1s, 'f2': f2s, 'f3': f3s, 'f4': f4s, 'f5': f5s, 'f6': f6s, 'f7': f7s, 'auc': scores}).sort_values(by=['auc'])
auc_csv.to_csv('freq_auc.csv')
auc_csv

,f0,f1,f2,f3,f4,f5,f6,f7,auc
0,0,0,0,0,0,0,0,0,0.000000
2,0,0,0,0,0,0,1,0,0.882728
1,0,0,0,0,0,0,0,1,0.889711
3,0,0,0,0,0,0,1,1,0.895471
5,0,0,0,0,0,1,0,1,0.980764
4,0,0,0,0,0,1,0,0,0.980828
6,0,0,0,0,0,1,1,0,0.981217
7,0,0,0,0,0,1,1,1,0.982000
11,0,0,0,0,1,0,1,1,0.982529
18,0,0,0,1,0,0,1,0,0.982658


In [15]:
print(score)

(0.992238129804707, 1, 1, 1, 1, 0, 0, 0, 0)


In [16]:
for i in range(8):
    feature_selected[i] = bool(score[i + 1])

XTrain, XTest = select_features(feature_selected)

## Try Others

In [5]:
feature_selected = [True,True,True,False,True,False,False,False,True] 
XTrain, XTest = select_features(feature_selected)

## Train models

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(XTrain, Y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(14929, 69159)
(3733, 69159)
(14929,)
(3733,)


In [7]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train, y_train)
valid_set = lgb.Dataset(X_valid, y_valid)

from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(XTrain, Y)
modelSelection = SelectFromModel(clf, prefit=True, max_features=8000)
XTrain = modelSelection.transform(XTrain)
XTrain.shape

XTest = modelSelection.transform(XTest)
XTest.shape

In [8]:
from hyperopt import hp
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import tpe
import lightgbm as lgb
from hyperopt import STATUS_OK

ITER = 50
STOP_ROUND = 5

# Create the dataset
def objective(params):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    params['num_leaves'] = int(params['num_leaves'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    bst = lgb.train(params, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
    bst.save_model('model.txt', num_iteration=bst.best_iteration)
  
    # Extract the best score
    best_score = bst.best_score['valid_0']['auc']
    
    # Loss must be minimized
    loss = np.log(1 - best_score)
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

# Define the search space
space = {
    'boosting_type': 'dart',
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'metric': 'auc'
}

# Trials object to track progress
bayes_trials = Trials()

MAX_EVALS = 500

# Optimize
bestDict = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials,
            rstate=np.random.RandomState(1024) # fixing random state for the reproducibility
           )
bestDict

[1]	valid_0's auc: 0.951146                                                                                            
  0%|                                                                            | 0/500 [00:07<?, ?it/s, best loss: ?]

c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\site-packages\lightgbm\callback.py:192: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



[2]	valid_0's auc: 0.964197                                                                                            
[3]	valid_0's auc: 0.965252                                                                                            
[4]	valid_0's auc: 0.967772                                                                                            
[5]	valid_0's auc: 0.968121                                                                                            
[6]	valid_0's auc: 0.968522                                                                                            
[7]	valid_0's auc: 0.968879                                                                                            
[8]	valid_0's auc: 0.968673                                                                                            
[9]	valid_0's auc: 0.970871                                                                                            
[10]	valid_0's auc: 0.971095            

[20]	valid_0's auc: 0.984455                                                                                           
[21]	valid_0's auc: 0.984379                                                                                           
[22]	valid_0's auc: 0.984723                                                                                           
[23]	valid_0's auc: 0.984921                                                                                           
[24]	valid_0's auc: 0.985324                                                                                           
[25]	valid_0's auc: 0.985487                                                                                           
[26]	valid_0's auc: 0.985681                                                                                           
[27]	valid_0's auc: 0.98576                                                                                            
[28]	valid_0's auc: 0.985793            

[38]	valid_0's auc: 0.980946                                                                                           
[39]	valid_0's auc: 0.98124                                                                                            
[40]	valid_0's auc: 0.981461                                                                                           
[41]	valid_0's auc: 0.981421                                                                                           
[42]	valid_0's auc: 0.981912                                                                                           
[43]	valid_0's auc: 0.981885                                                                                           
[44]	valid_0's auc: 0.982022                                                                                           
[45]	valid_0's auc: 0.982336                                                                                           
[46]	valid_0's auc: 0.982325            

...

[4]	valid_0's auc: 0.982276                                                                                            
[5]	valid_0's auc: 0.983465                                                                                            
[6]	valid_0's auc: 0.983093                                                                                            
[7]	valid_0's auc: 0.983689                                                                                            
[8]	valid_0's auc: 0.983874                                                                                            
[9]	valid_0's auc: 0.984732                                                                                            
[10]	valid_0's auc: 0.984661                                                                                           
[11]	valid_0's auc: 0.984862                                                                                           
[12]	valid_0's auc: 0.984876            

[22]	valid_0's auc: 0.989209                                                                                           
[23]	valid_0's auc: 0.989278                                                                                           
[24]	valid_0's auc: 0.989201                                                                                           
[25]	valid_0's auc: 0.989395                                                                                           
[26]	valid_0's auc: 0.989489                                                                                           
[27]	valid_0's auc: 0.989623                                                                                           
[28]	valid_0's auc: 0.989753                                                                                           
[29]	valid_0's auc: 0.989769                                                                                           
[30]	valid_0's auc: 0.989909            

[40]	valid_0's auc: 0.988221                                                                                           
[41]	valid_0's auc: 0.988213                                                                                           
[42]	valid_0's auc: 0.98849                                                                                            
[43]	valid_0's auc: 0.988524                                                                                           
[44]	valid_0's auc: 0.98855                                                                                            
[45]	valid_0's auc: 0.98855                                                                                            
[46]	valid_0's auc: 0.988758                                                                                           
[47]	valid_0's auc: 0.98876                                                                                            
[48]	valid_0's auc: 0.988596            

[8]	valid_0's auc: 0.978395                                                                                            
[9]	valid_0's auc: 0.97868                                                                                             
[10]	valid_0's auc: 0.979222                                                                                           
[11]	valid_0's auc: 0.97987                                                                                            
[12]	valid_0's auc: 0.979879                                                                                           
[13]	valid_0's auc: 0.980282                                                                                           
[14]	valid_0's auc: 0.980475                                                                                           
[15]	valid_0's auc: 0.981134                                                                                           
[16]	valid_0's auc: 0.981454            

{'colsample_by_tree': 0.6101535421906288,
 'learning_rate': 0.19202928417915138,
 'min_child_samples': 25.0,
 'num_leaves': 130.0,
 'reg_alpha': 0.1367833641980119,
 'reg_lambda': 0.99973269788873,
 'subsample_for_bin': 80000.0}

In [9]:
from hyperopt import space_eval
best = space_eval(space, bestDict)
best

{'boosting_type': 'dart',
 'colsample_bytree': 0.6101535421906288,
 'learning_rate': 0.19202928417915138,
 'metric': 'auc',
 'min_child_samples': 25.0,
 'num_leaves': 130.0,
 'reg_alpha': 0.1367833641980119,
 'reg_lambda': 0.99973269788873,
 'subsample_for_bin': 80000.0}

In [10]:
best['num_leaves'] = int(best['num_leaves'])
best['subsample_for_bin'] = int(best['subsample_for_bin'])
best['min_child_samples'] = int(best['min_child_samples'])
best['metric'] = 'auc'
lgbModel = lgb.train(best, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
lgbModel.save_model('model.txt', num_iteration=lgbModel.best_iteration)

# Extract the best score
best_score = lgbModel.best_score['valid_0']['auc']

# Loss must be minimized
loss = np.log(1 - best_score)
print(loss)
YTest = lgbModel.predict(XTest)

[1]	valid_0's auc: 0.97451
[2]	valid_0's auc: 0.981003
[3]	valid_0's auc: 0.983688
[4]	valid_0's auc: 0.98489
[5]	valid_0's auc: 0.986431
[6]	valid_0's auc: 0.986614
[7]	valid_0's auc: 0.987612
[8]	valid_0's auc: 0.987998
[9]	valid_0's auc: 0.98806
[10]	valid_0's auc: 0.988914
[11]	valid_0's auc: 0.989147
[12]	valid_0's auc: 0.98924
[13]	valid_0's auc: 0.989592
[14]	valid_0's auc: 0.989687
[15]	valid_0's auc: 0.989912
[16]	valid_0's auc: 0.990113
[17]	valid_0's auc: 0.990236
[18]	valid_0's auc: 0.990809
[19]	valid_0's auc: 0.991107
[20]	valid_0's auc: 0.991239
[21]	valid_0's auc: 0.991253
[22]	valid_0's auc: 0.991443
[23]	valid_0's auc: 0.991641
[24]	valid_0's auc: 0.991694
[25]	valid_0's auc: 0.991773
[26]	valid_0's auc: 0.991908
[27]	valid_0's auc: 0.991938
[28]	valid_0's auc: 0.991935
[29]	valid_0's auc: 0.991977
[30]	valid_0's auc: 0.991972
[31]	valid_0's auc: 0.99192
[32]	valid_0's auc: 0.991963
[33]	valid_0's auc: 0.992025
[34]	valid_0's auc: 0.991837
[35]	valid_0's auc: 0.991773

In [11]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(boosting_type=best['boosting_type'],
                        num_leaves=best['num_leaves'],
                        learning_rate=best['learning_rate'],
                        subsample_for_bin=best['subsample_for_bin'],
                        min_child_samples=best['min_child_samples'],
                        reg_alpha=best['reg_alpha'],
                        reg_lambda=best['reg_lambda'],
                        colsample_bytree=best['colsample_bytree'])
model.fit(XTrain, Y)
probs = model.predict_proba(XTest, num_iteration=lgbModel.best_iteration)

In [12]:
YTest = probs[:, 0]

In [13]:
df_test['Predicted'] = YTest
df_test

,Id,Predicted
0,0,0.000704
1,1,0.009538
2,2,0.998957
3,3,0.017557
4,4,0.478885
5,5,0.967080
6,6,0.999333
7,7,0.930065
8,8,0.000704
9,9,0.000541


In [14]:
df_test.to_csv('test.csv', index=False)